In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath=os.path.sep.join([r'C:\Users\PRANAY\Desktop\dataset','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\PRANAY\Desktop\dataset','res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
prototxtPath

'C:\\Users\\PRANAY\\Desktop\\dataset\\deploy.prototxt'

In [4]:
weightsPath

'C:\\Users\\PRANAY\\Desktop\\dataset\\res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [6]:
model=load_model(r'C:\Users\PRANAY\.keras\models\mobilenet_v2.model')

In [18]:
image=cv2.imread(r'Downloads\2.jpg')

In [19]:
image

array([[[117,  88,  97],
        [139, 110, 119],
        [135, 106, 115],
        ...,
        [181, 142, 150],
        [181, 142, 150],
        [181, 142, 150]],

       [[117,  88,  97],
        [139, 110, 119],
        [135, 106, 115],
        ...,
        [181, 142, 150],
        [181, 142, 150],
        [181, 142, 150]],

       [[119,  88,  97],
        [142, 111, 120],
        [137, 106, 115],
        ...,
        [182, 143, 151],
        [182, 143, 151],
        [182, 143, 151]],

       ...,

       [[  8, 102,   0],
        [ 11, 106,   0],
        [ 19, 110,   1],
        ...,
        [189, 174, 205],
        [173, 162, 194],
        [201, 192, 225]],

       [[  8, 102,   0],
        [ 12, 107,   0],
        [ 20, 111,   2],
        ...,
        [140, 129, 162],
        [127, 119, 156],
        [169, 164, 203]],

       [[  9, 103,   0],
        [ 12, 107,   0],
        [ 20, 111,   2],
        ...,
        [172, 162, 198],
        [151, 146, 185],
        [190, 186, 227]]

In [20]:
(h,w)=image.shape[:2]

In [21]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [22]:
blob

array([[[[  13.,   35.,   31., ...,   77.,   77.,   77.],
         [  13.,   35.,   31., ...,   77.,   77.,   77.],
         [  13.,   35.,   31., ...,   77.,   77.,   77.],
         ...,
         [ -96.,  -92.,  -84., ...,   39.,   25.,   67.],
         [ -96.,  -92.,  -84., ...,   57.,   39.,   79.],
         [ -95.,  -92.,  -84., ...,   68.,   47.,   86.]],

        [[ -89.,  -67.,  -71., ...,  -35.,  -35.,  -35.],
         [ -89.,  -67.,  -71., ...,  -35.,  -35.,  -35.],
         [ -89.,  -67.,  -71., ...,  -35.,  -35.,  -35.],
         ...,
         [ -75.,  -70.,  -66., ...,  -45.,  -55.,  -11.],
         [ -75.,  -70.,  -66., ...,  -26.,  -40.,    1.],
         [ -74.,  -70.,  -66., ...,  -15.,  -31.,    9.]],

        [[ -26.,   -4.,   -8., ...,   27.,   27.,   27.],
         [ -26.,   -4.,   -8., ...,   27.,   27.,   27.],
         [ -26.,   -4.,   -8., ...,   27.,   27.,   27.],
         ...,
         [-123., -123., -121., ...,   43.,   36.,   82.],
         [-123., -123., -1

In [23]:
net.setInput(blob)
detections=net.forward()

In [24]:
detections

array([[[[0.        , 1.        , 0.93999887, ..., 0.11296932,
          0.5602714 , 0.57658684],
         [0.        , 1.        , 0.10509964, ..., 0.61960024,
          0.13555464, 0.7919404 ],
         [0.        , 1.        , 0.09798207, ..., 0.46279895,
          0.2872632 , 0.9840516 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [27]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    if confidence>0.5:
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX),min(h-1,endY))
        
        face=image[startY:endY,startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        label="{}:{:.2f}%".format(label,max(mask,withoutMask*100))
        
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        cv2.imshow("output",image)
        cv2.waitKey(0)
        cv2.destroyAllWindows
        